In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 
from datetime import timedelta
%matplotlib inline

#from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score

import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

from imblearn.over_sampling import RandomOverSampler


In [2]:
labels = pd.read_csv("../../data/fiuba-trocafone-tp2-final-set/labels_training_set.csv")
df = pd.read_csv("../../data/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv")

C:\Users\seba\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_persons = pd.read_csv("../../data/trocafone_kaggle_test.csv")

In [4]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["month"] = df["timestamp"].dt.month
df["day"] = df["timestamp"].dt.day

In [5]:
todas_las_personas = df[["person"]].drop_duplicates(subset="person")

# Cantidad de veces que realiza un evento dado

In [192]:
cant_por_evento = pd.concat([pd.get_dummies(df['event']),df[['person']]],axis = 1).groupby('person')\
    .sum().reset_index()

df_train = pd.merge(cant_por_evento, labels, on="person", how="inner")\
            .drop(columns = ['person', 'viewed product', 'visited site', 'staticpage', 'lead'])
df_train.head()

,ad campaign hit,brand listing,checkout,conversion,generic listing,search engine hit,searched products,label
0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0
1,1.0,0.0,1.0,0.0,1.0,1.0,9.0,0
2,5.0,0.0,1.0,0.0,4.0,0.0,4.0,0
3,29.0,165.0,15.0,2.0,28.0,13.0,11.0,0
4,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0


In [193]:
#veo cuantos users pierdo
cant_por_evento_predecir = pd.merge(cant_por_evento, df_persons, on="person", how="inner")
cant_por_evento_predecir["person"].count() - df_persons["person"].count()

0

In [194]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [195]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [196]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [197]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 10, min_child_weight=1)


In [198]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=10, seed=None,
       silent=True, subsample=1)

In [199]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [200]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8537763183310798
Test acuracy:  0.8503734226113829
ROC auc score:  0.8260601668812715
Confusion matrix: 
[[3198  494]
 [  87  104]]


# Cantidad de veces que realiza un evento dado en el mes 5

In [212]:
eventos_mes_cinco = df.loc[df["month"] == 5]
cant_por_evento_mes_cinco = pd.concat([pd.get_dummies(eventos_mes_cinco['event']),eventos_mes_cinco[['person']]],axis = 1).groupby('person')\
    .sum().reset_index()

cant_por_evento_mes_cinco.columns = ['person', 'ad campaign hit mes 5', 'brand listing mes 5', 'checkout mes 5',\
                                     'conversion mes 5', 'generic listing mes 5', 'lead mes 5', 'search engine hit mes 5',\
                                     'searched products mes 5', 'staticpage mes 5', 'viewed product mes 5', 'visited site mes 5' ]

cant_por_evento_mes_cinco = pd.merge(cant_por_evento_mes_cinco, todas_las_personas, on="person", how='right')

df_train = pd.merge(cant_por_evento_mes_cinco, labels, on="person", how="inner")\
            .drop(columns = ['person', 'viewed product mes 5', 'visited site mes 5', 'staticpage mes 5', 'lead mes 5'])
cant_por_evento_mes_cinco.head()

,person,ad campaign hit mes 5,brand listing mes 5,checkout mes 5,conversion mes 5,generic listing mes 5,lead mes 5,search engine hit mes 5,searched products mes 5,staticpage mes 5,viewed product mes 5,visited site mes 5
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0
2,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0
3,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0
4,000e4d9e,19.0,17.0,1.0,0.0,17.0,0.0,5.0,0.0,0.0,339.0,13.0


In [314]:
df_combino = pd.merge(cant_por_evento_mes_cinco, cant_por_evento, on='person', how='inner')

df_train = pd.merge(df_combino, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

In [315]:
#veo cuantos users pierdo
cant_por_evento_predecir = pd.merge(cant_por_evento_mes_cinco, df_persons, on="person", how="inner")
cant_por_evento_predecir["person"].count() - df_persons["person"].count()

0

In [316]:
df_train.head()

,ad campaign hit mes 5,brand listing mes 5,checkout mes 5,conversion mes 5,generic listing mes 5,search engine hit mes 5,searched products mes 5,viewed product mes 5,visited site mes 5,ad campaign hit,brand listing,checkout,conversion,generic listing,search engine hit,searched products,label
0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0
1,1.0,0.0,1.0,0.0,1.0,1.0,9.0,3.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,9.0,0
2,5.0,0.0,1.0,0.0,4.0,0.0,4.0,4.0,1.0,5.0,0.0,1.0,0.0,4.0,0.0,4.0,0
3,16.0,113.0,4.0,1.0,13.0,7.0,1.0,123.0,12.0,29.0,165.0,15.0,2.0,28.0,13.0,11.0,0
4,0.0,1.0,2.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0


In [317]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [318]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [319]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [360]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=17)


In [361]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=17, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [362]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [363]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9083124074431782
Test acuracy:  0.8956992016482102
ROC auc score:  0.8517455322306225
Confusion matrix: 
[[3391  279]
 [ 126   87]]


## Cantidad de eventos en un dia del mes

In [364]:
df_day = pd.concat([df['person'],pd.get_dummies(df['day'])],axis = 1).groupby('person').sum().\
    reset_index()

In [365]:
columnas = ['dia_' + str(i) for i in range(1,32)]
columnas.insert(0,'person')
df_day.columns = columnas

In [366]:
df_con_dias_del_mes = pd.merge(df_combino, df_day, on='person', how='inner')

df_train = pd.merge(df_con_dias_del_mes, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

In [367]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [368]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [369]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [370]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=17)


In [371]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=17, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [372]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [373]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9057369132702338
Test acuracy:  0.8969868658253928
ROC auc score:  0.8561512581392077
Confusion matrix: 
[[3395  275]
 [ 125   88]]


## Cantidad de eventos en top K semanas del año.

In [431]:
K = 10
df['week_of_year'] = df['timestamp'].dt.weekofyear

In [432]:
df_week_of_year = df[['person','week_of_year']]

top_K_week_of_year = df_week_of_year['week_of_year'].value_counts().head(K).index

df_week_of_year = pd.concat([df_week_of_year,pd.get_dummies(df_week_of_year['week_of_year'])[top_K_week_of_year]],axis = 1)

df_week_of_year.drop(columns = ['week_of_year'],inplace = True)

df_week_of_year1 = df_week_of_year.groupby('person').sum().reset_index()

In [433]:
df_con_top_semanas = pd.merge(df_con_dias_del_mes, df_week_of_year1, on='person', how='inner')

df_train = pd.merge(df_con_top_semanas, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

In [444]:
df_train.head()

,ad campaign hit mes 5,brand listing mes 5,checkout mes 5,conversion mes 5,generic listing mes 5,search engine hit mes 5,searched products mes 5,viewed product mes 5,visited site mes 5,ad campaign hit,...,20,22,19,18,17,16,15,14,13,label
0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1.0,0.0,1.0,0.0,1.0,1.0,9.0,3.0,1.0,1.0,...,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,5.0,0.0,1.0,0.0,4.0,0.0,4.0,4.0,1.0,5.0,...,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,16.0,113.0,4.0,1.0,13.0,7.0,1.0,123.0,12.0,29.0,...,181.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,1.0,2.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [535]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [536]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [537]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [538]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=17)


In [539]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=17, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [540]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [544]:
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, proba)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.9067671109394115
Test acuracy:  0.8954416688127737

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      3670
           1       0.24      0.41      0.30       213

   micro avg       0.90      0.90      0.90      3883
   macro avg       0.60      0.67      0.62      3883
weighted avg       0.92      0.90      0.91      3883


Precision Score:  0.23848238482384823
Recall Score:  0.4131455399061033
F1 Score:  0.3024054982817869
Cohen Kappa Score:  0.25025490889860336
ROC auc score:  0.8574055596065037
Confusion matrix: 
[[3389  281]
 [ 125   88]]


# Usando random oversampling

In [516]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1].fillna(0), df_train.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [517]:
cant_ceros = np.sum(y_train == 0)
cant_unos = np.sum(y_train == 1)
cant_ceros, cant_unos

(14764, 767)

In [518]:
np.any(np.isnan(X_train))

False

In [519]:
np.all(np.isfinite(X))

True

In [520]:
ros = RandomOverSampler(random_state=2, sampling_strategy = {0: 14764, 1: 1200})
X_train_res, y_train_res = ros.fit_sample(X_train, y_train)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [529]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6,eval_metric = "auc", scale_pos_weight=5, min_child_weight=1)

In [530]:
#Entrenamos

xg_reg.fit(X_train_res_df ,y_train_res.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=5,
       seed=None, silent=True, subsample=1)

In [531]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [534]:
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, proba)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.8484715610122776
Test acuracy:  0.8411022405356683

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.85      0.91      3670
           1       0.20      0.65      0.31       213

   micro avg       0.84      0.84      0.84      3883
   macro avg       0.59      0.75      0.61      3883
weighted avg       0.93      0.84      0.88      3883


Precision Score:  0.20381231671554254
Recall Score:  0.6525821596244131
F1 Score:  0.3106145251396648
Cohen Kappa Score:  0.24772470580921035
ROC auc score:  0.8582332322728377
Confusion matrix: 
[[3127  543]
 [  74  139]]


## Visitas a la pagina segun resolucion

In [671]:
df_resolucion = df.loc[df['event'] == 'visited site']
df_resolucion = df_resolucion[['person','screen_resolution']].dropna()

top_5_resoluciones = df_resolucion['screen_resolution'].value_counts().head(10).index
df_resolucion = df_resolucion.loc[df_resolucion['screen_resolution'].isin(top_5_resoluciones)]
df_resolucion = pd.concat([df_resolucion['person'],\
        pd.get_dummies(df_resolucion['screen_resolution'])],axis = 1).groupby('person').sum().reset_index()

df_resolucion = pd.merge(df_resolucion, todas_las_personas, on="person", how="right").fillna(0)

df_resolucion.head()

,person,1024x768,1360x768,1366x768,1440x900,1600x900,1920x1080,320x534,320x568,320x570,360x640
0,0008ed71,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1,00091926,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,000ba417,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [672]:
# Creo que es mucho 20 resoluciones, probar (quizas empeore)..

In [673]:
# Veo cuantas personas a predecir pierdo
df_resolucion_predecir = pd.merge(df_resolucion, df_persons, on="person", how="inner")
df_resolucion_predecir["person"].count() - df_persons["person"].count()

0

In [675]:
# Merge

df_con_resoluciones = pd.merge(df_con_top_semanas, df_resolucion, on="person", how="inner") 
df_train = pd.merge(df_con_resoluciones, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])


In [676]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [677]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [678]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [715]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)


In [716]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=12, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [717]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [718]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9311055308737364
Test acuracy:  0.9175894926603142
ROC auc score:  0.8584545419656907
Confusion matrix: 
[[3500  170]
 [ 150   63]]


### Usando random oversampling

In [719]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1].fillna(0), df_train.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [720]:
cant_ceros = np.sum(y_train == 0)
cant_unos = np.sum(y_train == 1)
cant_ceros, cant_unos

(14764, 767)

In [721]:
np.any(np.isnan(X_train))

False

In [722]:
np.all(np.isfinite(X))

True

In [723]:
ros = RandomOverSampler(random_state=2, sampling_strategy = {0: 14764, 1: 1200})
X_train_res, y_train_res = ros.fit_sample(X_train, y_train)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [751]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6,eval_metric = "auc", scale_pos_weight=5, min_child_weight=20)

In [752]:
#Entrenamos

xg_reg.fit(X_train_res_df ,y_train_res.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=20, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=5,
       seed=None, silent=True, subsample=1)

In [753]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [742]:
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, proba)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.8737158606865447
Test acuracy:  0.871233582281741

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.89      0.93      3670
           1       0.23      0.58      0.33       213

   micro avg       0.87      0.87      0.87      3883
   macro avg       0.60      0.73      0.63      3883
weighted avg       0.93      0.87      0.90      3883


Precision Score:  0.23076923076923078
Recall Score:  0.5774647887323944
F1 Score:  0.3297587131367293
Cohen Kappa Score:  0.2727538338215353
ROC auc score:  0.860972739251129
Confusion matrix: 
[[3260  410]
 [  90  123]]


OCHENTA Y SEIS

# Distancia entre ultima entrada al sitio y el ultimo dia que tenemos

In [925]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia_de_todos = timestamp_df["timestamp"].max()
ultima_entrada = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
ultima_entrada["dias_hasta_ultimo"] = (ultimo_dia_de_todos - ultima_entrada["ultimo"]).dt.days
ultima_entrada = ultima_entrada[["person", "dias_hasta_ultimo"]]
ultima_entrada.head()

,person,dias_hasta_ultimo
0,0008ed71,14
1,00091926,0
2,00091a7a,66
3,000ba417,5
4,000c79fe,2


In [926]:
# Veo cuantas personas a predecir pierdo
df_predecir = pd.merge(ultima_entrada, df_persons, on="person", how="inner")
df_predecir["person"].count() - df_persons["person"].count()

0

In [1051]:
# Merge
# df_con_resoluciones

df_con_diatancia_hasta_ultimo = pd.merge(df_con_resoluciones, ultima_entrada, on="person", how="inner") 
df_train = pd.merge(df_con_diatancia_hasta_ultimo, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])


In [1052]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [1053]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [1054]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1055]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)


In [1056]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=12, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [1057]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [1058]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9307835941021183
Test acuracy:  0.921452485191862
ROC auc score:  0.8602141459108877
Confusion matrix: 
[[3508  162]
 [ 143   70]]


### Usando random oversampling

In [1060]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1].fillna(0), df_train.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1061]:
cant_ceros = np.sum(y_train == 0)
cant_unos = np.sum(y_train == 1)
cant_ceros, cant_unos

(14764, 767)

In [1062]:
ros = RandomOverSampler(random_state=2, sampling_strategy = {0: 14764, 1: 784})
X_train_res, y_train_res = ros.fit_sample(X_train, y_train)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [1063]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6,eval_metric = "auc", scale_pos_weight=5, min_child_weight=10)

In [1064]:
#Entrenamos

xg_reg.fit(X_train_res_df ,y_train_res.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=10, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=5,
       seed=None, silent=True, subsample=1)

In [1065]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [1067]:
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, proba)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.928222279392848
Test acuracy:  0.9173319598248777

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.95      0.96      3670
           1       0.28      0.33      0.30       213

   micro avg       0.92      0.92      0.92      3883
   macro avg       0.62      0.64      0.63      3883
weighted avg       0.92      0.92      0.92      3883


Precision Score:  0.28225806451612906
Recall Score:  0.3286384976525822
F1 Score:  0.3036876355748373
Cohen Kappa Score:  0.2600143076379633
ROC auc score:  0.8628461961597013
Confusion matrix: 
[[3492  178]
 [ 143   70]]


 OCHENTA Y SEIS DOS

# -- Otros features que tal vez sirvan --

# Distancia en dias de 1er y ultimo dia de entrada al sitio

In [91]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
primer_dia = timestamp_df.groupby("person")["timestamp"].min().rename("primer").reset_index()
distancia_dias = pd.merge(primer_dia, ultimo_dia, on="person", how="inner")
distancia_dias["distan_dias"] = (distancia_dias["ultimo"] - distancia_dias["primer"]).dt.days
distancia_dias = distancia_dias[["person", "distan_dias"]]
distancia_dias.head()

,person,distan_dias
0,0008ed71,0
1,00091926,27
2,00091a7a,0
3,000ba417,9
4,000c79fe,0


In [596]:
# Merge
#df_con_mes_5
df_con_distancias = pd.merge(distancia_dias, df_con_top_semanas, on="person", how="inner") 
df_train = pd.merge(df_con_distancias, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

In [597]:
# Veo cuantas personas a predecir pierdo
distancia_dias_predecir = pd.merge(distancia_dias, df_persons, on="person", how="inner")
distancia_dias_predecir["person"].count() - df_persons["person"].count()

0

## Cantidad de modelos distintos vistos por cada usuario

In [545]:
modelos_df = df[["person", "model"]]
cant_modelos = modelos_df.groupby("person")["model"].nunique().rename("modelos_dist").reset_index()
cant_modelos.head()

,person,modelos_dist
0,0008ed71,3
1,00091926,36
2,00091a7a,3
3,000ba417,26
4,000c79fe,1


In [609]:
# Merge 
#df_con_distancias
df_con_modelos = pd.merge(df_con_top_semanas, cant_modelos, on="person", how="inner") 
df_train = pd.merge(df_con_modelos, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

In [610]:
# Veo cuantas personas a predecir pierdo
cant_modelos_predecir = pd.merge(cant_modelos, df_persons, on="person", how="inner")
cant_modelos_predecir["person"].count() - df_persons["person"].count()

0

## Cantidad de días distintos de entrada al sitio

In [121]:
df_dias = df[["person", "timestamp"]]
df_dias["day_of_year"] = df_dias["timestamp"].dt.dayofyear
cant_dias_dist = df_dias.groupby("person")["day_of_year"].nunique().rename("cant_dias_dist").reset_index()
cant_dias_dist.head()

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,person,cant_dias_dist
0,0008ed71,1
1,00091926,22
2,00091a7a,1
3,000ba417,3
4,000c79fe,1


In [637]:
# Merge df_con_modelos

df_con_dias_dist = pd.merge(df_con_top_semanas, cant_dias_dist, on="person", how="inner") 
df_train = pd.merge(df_con_dias_dist, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])


Index([  'ad campaign hit mes 5',     'brand listing mes 5',
                'checkout mes 5',        'conversion mes 5',
         'generic listing mes 5', 'search engine hit mes 5',
       'searched products mes 5',    'viewed product mes 5',
            'visited site mes 5',         'ad campaign hit',
                 'brand listing',                'checkout',
                    'conversion',         'generic listing',
             'search engine hit',       'searched products',
                         'dia_1',                   'dia_2',
                         'dia_3',                   'dia_4',
                         'dia_5',                   'dia_6',
                         'dia_7',                   'dia_8',
                         'dia_9',                  'dia_10',
                        'dia_11',                  'dia_12',
                        'dia_13',                  'dia_14',
                        'dia_15',                  'dia_16',
                        

In [619]:
# Veo cuantas personas a predecir pierdo
cant_dias_dist_predecir = pd.merge(cant_dias_dist, df_persons, on="person", how="inner")
cant_dias_dist_predecir["person"].count() - df_persons["person"].count()

0

# Entro como 'new' en el mes 5

In [653]:
df_mes_5 = df[["person", "month", "new_vs_returning"]].loc[df["month"] == 5]
df_mes_5["nuevo_mes_5"] = (df_mes_5["new_vs_returning"] == "New") * 1
df_mew_5 = df_mes_5[["person", "nuevo_mes_5"]]
df_mew_5 = df_mew_5.sort_values("nuevo_mes_5", ascending=False)
df_mew_5 = df_mew_5.drop_duplicates(subset=['person'], keep='first')
df_mew_5 = pd.merge(df_mew_5, todas_las_personas, on="person", how="right").fillna(0)
df_mew_5.head()

,person,nuevo_mes_5
0,c0f2beb1,1.0
1,9ff9d5d2,1.0
2,ae9e5fa3,1.0
3,bbbe368d,1.0
4,0b7a3edd,1.0


In [654]:
# Veo cuantas personas a predecir pierdo
df_mew_5_predecir = pd.merge(df_mew_5, df_persons, on="person", how="inner")
df_mew_5_predecir["person"].count() - df_persons["person"].count()

0

In [655]:
# Merge

df_con_new_mes5 = pd.merge(df_con_top_semanas, df_mew_5, on="person", how="inner") 
df_train = pd.merge(df_con_new_mes5, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])
df_train['nuevo_mes_5'].value_counts()

1.0    14789
0.0     4625
Name: nuevo_mes_5, dtype: int64